# Figure 3 Final

Notebook to make figure 3 for ms <br>
by Cascade Tuholske 2020.02.23

Updated 2020.04.08 by CPT to drop the scatter plots<br>
Ran 2020.09.08 by CPT for MS <br>

Updated Feb 2021 for PNASv2 CPT

In [1]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from scipy.optimize import fmin
from scipy.stats import beta
from scipy.special import gamma as gammaf
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import operator

In [2]:
def OLS(df, geog, col, alpha):
    
    """Finds linear coef for increase in stat by a given geography from 1983 - 2016, as well
    as the pct change in population of the cities within the given geography
    
    NOTE 2020.03.01 - This will throw a run time warning if all values of a col are zero (e.g. can regress
    a bunch of zeros) ... See note in run_OLS. CPT 
    
    NOTE 2020.03.01 - Later in the day this issue is resolved by removing the offending cities. See comments
    in code. CPT
    
    
    Args:
        df = HI stats dataframe
        geog = subset geography to calc people days regression
        col = col to regress on 
        alpha = ci alpha for coef
    """

    # Get results
    labels = []
    coef_list = []
    leftci_list = []
    rightci_list = []
    p_list = []
    df_out = pd.DataFrame()

    for label, df_geog in df.groupby(geog):

        # Get Data
        X_year = np.array(df_geog.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
        Y_stats = np.array(df_geog.groupby('year')[col].sum()).reshape((-1, 1))

        # Add Intercept
        X_year_2 = sm.add_constant(X_year)

        # Regress
        model = sm.OLS(Y_stats, X_year_2).fit() 
        
        # Get slope
        # first param in intercept coef, second is slope of line but if slope = 0, then intecept
        if len(model.params) == 2:
            coef = model.params[1]
            
        else:
            coef = model.params[0]
        
        # P value
        p = model.pvalues[0]

        # Make lists
        labels.append(label)

        coef_list.append(coef)
        p_list.append(p)

    # Make data frame
    df_out[geog] = labels

    df_out['coef'] = coef_list
    df_out['p_value'] = [round(elem, 4) for elem in p_list]

    return df_out


In [3]:
def run_OLS(stats, geog, alpha):
    """ Function calculate OLS coef of people days due to pop and heat and the 
    attribution index for distribution plots.
    
        
    NOTE 2020.03.01 - This will throw a run time warning if all values of a col are zero (e.g. can regress
    a bunch of zeros, now can we). This will happen if people_days, people_days_pop, people_days_heat or 
    total_days is zero for all years for a given city. This is still OK for our analysis. What is happening is
    that for some cities, the people-days due to heat is zero, meaning pday increases in only due to population. 
    
    This is because with the GHS-UCDB some city's population in 1983 is zero, which forces the pdays due to heat
    to be zero.
    
    NOTE 2020.03.01 - Later in the day this issue is resolved by removing the offending cities. See comments
    in code.
    
    -- CPT  
    
    Args:
        stats = df to feed in
        geog = geography level to conduct analysis (city-level is 'ID-HDC-G0')
        alpha = alpha for CI coef   
    """
    # Get coef for people days
    out = OLS(stats, geog, 'people_days', alpha = alpha)
    out.rename(columns={"coef": "coef_pdays"}, inplace = True)
    out.rename(columns={"p_value": "p_value_pdays"}, inplace = True)
    
    # Get people days due to heat coef
    heat = OLS(stats, geog, 'people_days_heat', alpha = alpha) # get stats 
    heat.rename(columns={"coef": "coef_heat"}, inplace = True)
    heat.rename(columns={"p_value": "p_value_heat"}, inplace = True)
    out = out.merge(heat, on = geog, how = 'left') # merge
    
    # Get people days due to pop
    pop = OLS(stats, geog, 'people_days_pop', alpha = alpha) # get stats 
    pop.rename(columns={"coef": "coef_pop"}, inplace = True)
    pop.rename(columns={"p_value": "p_value_pop"}, inplace = True)
    out = out.merge(pop, on = geog, how = 'left') # merge
    
    # Get total days
    totDays = OLS(stats, geog, 'total_days', alpha = alpha) # get stats 
    totDays.rename(columns={"coef": "coef_totDays"}, inplace = True)
    totDays.rename(columns={"p_value": "p_value_totDays"}, inplace = True)
    out = out.merge(totDays, on = geog, how = 'left') # merge
    
    # attrib coef --- creates range -1 to 1 index of heat vs. population as a driver of total pdays increase
    out['coef_attrib'] = (out['coef_pop'] - out['coef_heat']) / (out['coef_pop'] + out['coef_heat']) # normalize dif
    
    # drop all neg or zero pday slopes (e.g. cooling cities)
    out = out[out['coef_pdays'] > 0]
    out = out[out['coef_heat'] > 0]
    out = out[out['coef_pop'] > 0]
    
    # normalize coef of attribution 
    norm = out['coef_attrib']
    out['coef_attrib_norm'] = (norm-min(norm))/(max(norm)-min(norm))
    
    return out

In [ ]:
#### Load Data
# file path
DATA_IN = "/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/"  # Note: Need ?dl=1 to make sure this file gets read correctly
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"

# Raw Heat
FN_IN = 'AllDATA-GHS-ERA5-HI406-PDAYS.csv'
HI_STATS = pd.read_csv(DATA_IN+FN_IN)

# File out
FN_OUT = 'FIG3_ERA5_Final-IRI.png'

# scale the date in the plot 
scale = 10**9 

print(HI_STATS.head(2))

# cols we want to add to HI_STATS
cols = ['region', 'intermediate-region', 'sub-region','CTR_MN_NM', 'ID_HDC_G0'] 

# open all the data
meta_fn = 'processed/AllDATA-GHS-ERA5-HI406-META.csv' # open all the data
all_data = pd.read_csv(DATA_IN+meta_fn)

# drop ID duplicates
meta = all_data.drop_duplicates('ID_HDC_G0')
meta = meta[cols]

HI_STATS = HI_STATS.merge(meta, on = 'ID_HDC_G0', how = 'inner')
print(HI_STATS.head(2))

In [ ]:
# Drop cites where 1983 had 1 day and none elsewhere

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['total_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['total_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

In [ ]:
#### Chunk Data for plots -- see functions for warnings explained 

# Get Populations
df_years = HI_STATS[['ID_HDC_G0', 'P1983', 'P2016']]
df_years = df_years.drop_duplicates('ID_HDC_G0')

# Global
# plotdata = run_OLS(HI_STATS, 'ID_HDC_G0', alpha = 0.05)

# Chunk 1
geog1 = 'region'
loc1 = 'Europe'
chunk1 = HI_STATS[HI_STATS[geog1] == loc1]
plotdata1 = run_OLS(chunk1, 'ID_HDC_G0', alpha = 0.05)
plotdata1 = plotdata1.merge(df_years, on = 'ID_HDC_G0', how = 'left')
print(len(plotdata1))
#plotdata1 = plotdata1[plotdata1['p_value_pdays'] < 0.05] # drop Pdays below 0.05
print(len(plotdata1))

# # Chunk 2
geog2 = 'sub-region'
loc2 = 'Latin America and the Caribbean'
chunk2 = HI_STATS[HI_STATS[geog2] == loc2]
plotdata2 = run_OLS(chunk2, 'ID_HDC_G0', alpha = 0.05)
plotdata2 = plotdata2.merge(df_years, on = 'ID_HDC_G0', how = 'left')
print(len(plotdata2))
plotdata2 = plotdata2[plotdata2['p_value_pdays'] < 0.05] # drop Pdays below 0.05
print(len(plotdata2))

# # Chunk 3
geog3 = 'sub-region'
loc3 = 'Southern Asia'
chunk3 = HI_STATS[HI_STATS[geog3] == loc3]
plotdata3 = run_OLS(chunk3, 'ID_HDC_G0', alpha = 0.05)
plotdata3 = plotdata3.merge(df_years, on = 'ID_HDC_G0', how = 'left')
print(len(plotdata3))
plotdata3 = plotdata3[plotdata3['p_value_pdays'] < 0.05] # drop Pdays below 0.05
print(len(plotdata3))

# Chunk 4
geog4 = 'sub-region'
loc4 = 'Sub-Saharan Africa'
chunk4 = HI_STATS[HI_STATS[geog4] == loc4]
plotdata4 = run_OLS(chunk4, 'ID_HDC_G0', alpha = 0.05)
plotdata4 = plotdata4.merge(df_years, on = 'ID_HDC_G0', how = 'left')
print(len(plotdata4))
plotdata4 = plotdata4[plotdata4['p_value_pdays'] < 0.05] # drop Pdays below 0.05
print(len(plotdata4))


In [ ]:
#### Fig 3 Plot - 4 Panel CPT 2020.04.08

# Chunks in a list & colors
df_list = [plotdata1 , plotdata2 , plotdata3 , plotdata4]
color_list = ['purple', 'teal', 'dodgerblue', 'orange']
loc_list = [loc1, loc2, loc3, loc4]

# Size
fig, axs = plt.subplots(1, 4, figsize = (24, 6), sharex=False, sharey = False)
axs = axs.ravel()
fig.subplots_adjust(wspace= 0.25, hspace = 0.35)
fig = plt.gcf()

# Plot args
kwargs = {'lw': 3, 'alpha' : 0.9, 'linestyle' : '-'} # ked line kwargs

for i, df in enumerate(zip(df_list, color_list)):
    
    xlim = [0,1]
    ylim = [0,4]
    ticks = [0, .25, .5, .75, 1]
    #labels = [str(100), 50, 0, 50, str(100)]#[100, 80, 60, 40, 20, 0, 20, 40, 60, 80, 100]
    labels = ['100%','Contribution\nWarming', 'Equal', 'Contribution\nPopulation', '100%']

    # Git data for hist and poly fit 
    data = df[0]['coef_attrib_norm'] 
    axs[i].hist(data, density = True, color = df[1], alpha = 0.50, bins = 30) 
    x = axs[i].hist(data, density = True, color = df[1], alpha = 0, bins = 30)[1]
    y = axs[i].hist(data, density = True, color = df[1], alpha = 0, bins = 30)[0]
    y = np.insert(y, 0, y[0])

    # Poly fit plot
    x = x[:, np.newaxis]
    y = y[:, np.newaxis]
    polynomial_features= PolynomialFeatures(degree=4)
    x_poly = polynomial_features.fit_transform(x)
    model = LinearRegression()
    model.fit(x_poly, y)
    y_poly_pred = model.predict(x_poly)
    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
    x, y_poly_pred = zip(*sorted_zip)
    axs[i].plot(x, y_poly_pred, alpha = 0.9, linewidth=3, color = color_list[i])

    # V line 
    axs[i].vlines(x = 0.5, ymin = 0, ymax = 5, lw=2, linestyles = ':', 
                  alpha = 0.5, color = 'black')

    #sns.kdeplot(data, color = df[1], ax = axs[i], **kwargs, legend = False) # kernel
    axs[i].set_title(loc_list[i], fontsize = 15) 
    axs[i].set_xlim(xlim)
    axs[i].set_ylim(ylim)
    axs[i].set_ylabel('PDF', fontsize = 15)
    #axs[i].set_xlabel('% Warming                 % Population', fontsize = 15)
    axs[i].set_xticks(ticks);
    axs[i].set_xticklabels(labels)
    
    # add panel labels
    axs[0].text(0.00, 1.1, '(A)', transform=axs[0].transAxes, size=15, fontweight='bold')
    axs[0].text(1.25, 1.1, '(B)', transform=axs[0].transAxes, size=15, fontweight='bold')
    axs[0].text(2.5, 1.1, '(C)', transform=axs[0].transAxes, size=15, fontweight='bold')
    axs[0].text(3.75, 1.1, '(D)', transform=axs[0].transAxes, size=15, fontweight='bold')
    
### color bar on far left plot
kwargs = {'anchor' : (-0.5, 0.5), 'fraction' : .01, 'shrink' : 2}
fig.subplots_adjust(right=0.84)

plt.savefig(FIG_OUT+FN_OUT, dpi = 300, bbox_inches='tight')

# More panels

In [ ]:
#### Fig 3 Plot

# Chunks in a list & colors
df_list = [plotdata1 , plotdata2 , plotdata3 , plotdata4, plotdata1 , plotdata2 , plotdata3 , plotdata4]
color_list = ['purple', 'teal', 'dodgerblue', 'orange', 'purple', 'teal', 'dodgerblue', 'orange']
loc_list = [loc1, loc2, loc3, loc4, loc1, loc2, loc3, loc4]

# Size
fig, axs = plt.subplots(2, 4, figsize = (24, 12), sharex=False, sharey = False)
axs = axs.ravel()
fig.subplots_adjust(wspace= 0.25, hspace = 0.35)
fig = plt.gcf()

# Plot args
kwargs = {'lw': 3, 'alpha' : 0.9, 'linestyle' : '-'} # ked line kwargs

for i, df in enumerate(zip(df_list, color_list)):
    
    # First row
    if i < 4:
        xlim = [0,1]
        ylim = [0,4]
        ticks = [0, .25, .5, .75, 1]
        #labels = [str(100), 50, 0, 50, str(100)]#[100, 80, 60, 40, 20, 0, 20, 40, 60, 80, 100]
        labels = ['100%','Contribution\nWarming', 'Equal', 'Contribution\nPopulation', '100%']
        
        # Git data for hist and poly fit 
        data = df[0]['coef_attrib_norm'] 
        axs[i].hist(data, density = True, color = df[1], alpha = 0.15, bins = 30) 
        x = axs[i].hist(data, density = True, color = df[1], alpha = 0, bins = 30)[1]
        y = axs[i].hist(data, density = True, color = df[1], alpha = 0, bins = 30)[0]
        y = np.insert(y, 0, y[0])
        
        # Poly fit plot
        x = x[:, np.newaxis]
        y = y[:, np.newaxis]
        polynomial_features= PolynomialFeatures(degree=4)
        x_poly = polynomial_features.fit_transform(x)
        model = LinearRegression()
        model.fit(x_poly, y)
        y_poly_pred = model.predict(x_poly)
        sort_axis = operator.itemgetter(0)
        sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
        x, y_poly_pred = zip(*sorted_zip)
        axs[i].plot(x, y_poly_pred, alpha = 0.9, linewidth=3, color = color_list[i])
        
        # V line 
        axs[i].vlines(x = 0.5, ymin = 0, ymax = 5, lw=2, linestyles = ':', 
                      alpha = 0.5, color = 'black')
        
        #sns.kdeplot(data, color = df[1], ax = axs[i], **kwargs, legend = False) # kernel
        axs[i].set_title(loc_list[i], fontsize = 15) 
        axs[i].set_xlim(xlim)
        axs[i].set_ylim(ylim)
        axs[i].set_ylabel('PDF', fontsize = 15)
        #axs[i].set_xlabel('% Warming                 % Population', fontsize = 15)
        axs[i].set_xticks(ticks);
        axs[i].set_xticklabels(labels)
    
    # Second row
    if i >= 4:
        lim = 8
        
        # Make Data
        X = np.log10(df[0]['coef_pop'])
        Y = np.log10(df[0]['coef_heat'])
        C = np.log10(df[0]['P2016']) 

        # plots
        im  = axs[i].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

        # Make one-to-one-line
        axs[i].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
        axs[i].set_title(loc_list[i], fontsize = 15)
        
        if i == 4:
            cmap = axs[i].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

        #fig.colorbar(ax = axs[i],  mappable = cmap) color Bar all ploots

        # Set limits
        axs[i].set_xlim([-0.5, lim])
        axs[i].set_ylim([-0.5, lim])
        
        axs[i].set_ylabel('Increase from Warming [log10]', fontsize = 15)
        axs[i].set_xlabel('Increase from Population [log10]', fontsize = 15)

    # add panel labels
    axs[0].text(0.00, 1.1, '(a)', transform=axs[0].transAxes, size=15 )
    axs[0].text(1.25, 1.1, '(b)', transform=axs[0].transAxes, size=15 )
    axs[0].text(2.5, 1.1, '(c)', transform=axs[0].transAxes, size=15 )
    axs[0].text(3.75, 1.1, '(d)', transform=axs[0].transAxes, size=15 )
    axs[0].text(0.00, -.24, '(e)', transform=axs[0].transAxes, size=15 )
    axs[0].text(1.25, -.24, '(f)', transform=axs[0].transAxes, size=15 )
    axs[0].text(2.5, -.24, '(g)', transform=axs[0].transAxes, size=15 )
    axs[0].text(3.75, -.24, '(h)', transform=axs[0].transAxes, size=15 )

    
### color bar on far left plot
kwargs = {'anchor' : (-0.5, 0.5), 'fraction' : .01, 'shrink' : 2}
fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.12, 0.01, 0.33])
fig.colorbar(im, cax=cbar_ax, **kwargs)

# Set the ticks and ticklabels for all axes
#plt.setp(axs, xticks=ticks, xticklabels=labels);

## Labels 
# plt.xlabel('Coef of Attribtuion (Right is people, left heat)', fontsize = 15, labelpad = 10)
# plt.ylabel('PDF', fontsize = 15)
# fig.title('Dist of PeopleDays Attribution Coef, 1983 - 2016', fontsize = 15)
#plt.savefig(FIG_OUT+FNOUT, dpi = 300, bbox_inches='tight')

# Try poly fit

In [ ]:
data = plotdata1
title = loc1
Y = plt.hist(data['coef_attrib'], bins = 30)[0]
X = plt.hist(data['coef_attrib'], bins = 30)[1]
Y = np.insert(Y, 0, Y[0])

In [ ]:
import operator

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

np.random.seed(0)
x = X
y = Y

# transforming the data to include another axis
x = x[:, np.newaxis]
y = y[:, np.newaxis]

polynomial_features= PolynomialFeatures(degree=4)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)
print(rmse)
print(r2)

plt.hist(data['coef_attrib'], bins = 30, alpha = .2)
# sort the values of x before line plot
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.plot(x, y_poly_pred, color='m')
plt.title(title+' 4th order poly R2='+str(round(r2,2)))
plt.show()